Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give read access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [ ]:
## DONT CHANGE THIS CELL
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip

--2021-10-25 08:22:54--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip [following]
--2021-10-25 08:22:55--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217313 (212K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 212.22K   111KB/s    in 1.9s    

2021-10-25 08:22:58 (111 KB/s) - ‘data.zip’ saved [217313/217313]



In [ ]:
!pip3 install Sentencepiece
!pip3 install transformers

     |████████████████████████████████| 1.2 MB 5.4 MB/s 
     |████████████████████████████████| 2.9 MB 4.4 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 596 kB 48.1 MB/s 
     |████████████████████████████████| 895 kB 39.2 MB/s 
     |████████████████████████████████| 3.3 MB 18.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# **Imports**

In [ ]:
import pandas as pd
import pandas as pd
import math
import re
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
import numpy as np
import datetime
from torchtext.legacy import data as torchtext_data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe, FastText

from transformers import BertTokenizer, BertTokenizerFast, BertForSequenceClassification,BertConfig, BertModel
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# **Prepare data csv**

In [ ]:
!mkdir model
!unzip /content/data.zip

Archive:  /content/data.zip
   creating: data/
   creating: data/train/
  inflating: data/train/train.gold.txt  
  inflating: data/train/train.data.txt  
   creating: data/validation/
  inflating: data/validation/validation.data.txt  
  inflating: data/validation/validation.gold.txt  


In [ ]:
with open("/content/data/train/train.data.txt","r") as f:
  train_data = f.read()
with open("/content/data/train/train.gold.txt","r") as f:
  train_labels = f.readlines()

c=0
dataset = []
temp = {"sent1":"","sent2":"","word":"","pos1":0,"pos2":0,"label":0,"pos_tag":""}
for data in train_data.split("\n"):
  if(data != ''):
      temp = {"sent1":"","sent2":"","word":"","pos1":0,"pos2":0,"label":0}
      temp["text"] = data.split("\t")[3] + " [SEP] " + data.split("\t")[4]
      temp["sent1"] = data.split("\t")[3]
      temp["sent2"] = data.split("\t")[4]
      temp["pos1"] = int(data.split("\t")[2].split("-")[0])
      temp["pos2"] = int(data.split("\t")[2].split("-")[1])
      temp["word"] = data.split("\t")[0]
      temp["pos_tag"] = data.split("\t")[1]
      temp["label"] = 1 if (train_labels[c].strip() in ["T","t"]) else 0
      dataset.append(temp)
      c += 1

pd.DataFrame(dataset).to_csv("/content/train_data.tsv",sep="\t",index=False)


In [ ]:
with open("/content/data/validation/validation.data.txt","r") as f:
  validation_data = f.read()
with open("/content/data/validation/validation.gold.txt","r") as f:
  validation_labels = f.readlines()

c=0
dataset = []
temp = {"sent1":"","sent2":"","word":"","pos1":0,"pos2":0,"label":0}
for data in validation_data.split("\n"):
  if(data != ''):
      temp = {"sent1":"","sent2":"","word":"","pos1":0,"pos2":0,"label":0}
      temp["text"] = data.split("\t")[3] + " [SEP] " + data.split("\t")[4]
      temp["sent1"] = data.split("\t")[3]
      temp["sent2"] = data.split("\t")[4]
      temp["pos1"] = int(data.split("\t")[2].split("-")[0])
      temp["pos2"] = int(data.split("\t")[2].split("-")[1])
      temp["word"] = data.split("\t")[0]
      temp["pos_tag"] = data.split("\t")[1]
      temp["label"] = 1 if (validation_labels[c].strip() in ["T","t"]) else 0
      dataset.append(temp)
      c += 1

pd.DataFrame(dataset).to_csv("/content/validation_data.tsv",sep="\t",index=False)

# **NLTK WORDNET**

In [ ]:
from tqdm import tqdm
import pandas as pd
import itertools

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
from nltk.corpus import wordnet as wn

In [ ]:
from tqdm import tqdm
all_synset = []
for synset in tqdm(list(wn.all_synsets())):
    try:
        n = synset.name()
        all_synset.append({"name":n.split(".")[0], "pos":n.split(".")[1], "num":int(n.split(".")[2])})
    except Exception as e:
      pass

SyntaxError: ignored

In [ ]:
noun_synsets = pd.DataFrame(all_synset)

In [ ]:
noun_synsets.head()

,name,pos,num
68589,'hood,n,1
103017,1530s,n,1
65698,15_may_organization,n,1
103029,1750s,n,1
103033,1760s,n,1


In [ ]:
noun_synsets.sort_values(["name","pos","num"], inplace=True)

In [ ]:
noun_synsets.head()

,name,pos,num
68589,'hood,n,1
103017,1530s,n,1
65698,15_may_organization,n,1
103029,1750s,n,1
103033,1760s,n,1


In [ ]:
noun_synsets.drop_duplicates(["name"], keep="first", inplace=True)

In [ ]:
reg = re.compile(r"[^a-zA-Z]")

negative_samples = []
positive_samples = []
for row in tqdm(noun_synsets.to_dict("records")[:]):
    temp_neg = []
    if(True):
        for i in range(100):
            try:
                if(not reg.search(row["name"])):
                    examples = wn.synset(f'{row["name"]}.{row["pos"]}.{i}').examples()
                    pos_examples = [i for i in examples if row["name"] in i ]
                    if (len(pos_examples)) > 1:
                        s = list(itertools.permutations(pos_examples,2))
                        s = [tuple(sorted(i)) for i in s]
                        s = list(set(s))
                        positive_samples.extend(s)
                    for j in examples:
                        if(row["name"] in j):
                            temp_neg.append(j)
                            break
            except Exception as e:
                break
        s = []
        if(len(temp_neg)>2):
            s = list(itertools.permutations(temp_neg,2))[:]
            s = [tuple(sorted(i)) for i in s][:]
            s = list(set(s))
        if(s != []):
            negative_samples.extend(s)
            temp_neg = []

100%|██████████| 86547/86547 [00:02<00:00, 36583.13it/s]


In [ ]:
len(positive_samples), len(negative_samples)

(11881, 26437)

In [ ]:
positive_samples = [i for i in positive_samples if i[0]!=i[1]]
negative_samples = [i for i in negative_samples if i[0]!=i[1]]

In [ ]:
len(positive_samples), len(negative_samples)

(11881, 22786)

In [ ]:
ps = pd.DataFrame(positive_samples, columns=["sent1","sent2"])
ps["label"] = 1

In [ ]:
ps.head()

,sent1,sent2,label
0,abject poverty,the most abject slaves joined in the revolt,1
1,an able seaman,every able-bodied young man served in the army,1
2,able to swim,she was able to program her computer,1
3,she was able to program her computer,we were at last able to buy a car,1
4,able to get a grant for the project,we were at last able to buy a car,1


In [ ]:
ns = pd.DataFrame(negative_samples, columns=["sent1","sent2"])
ns["label"] = 0

In [ ]:
wordnet_data = pd.DataFrame()
wordnet_data = wordnet_data.append(ps)
wordnet_data = wordnet_data.append(ns)

In [ ]:
wordnet_data["text"] = wordnet_data.apply(lambda x : x["sent1"]+". [SEP] "+x["sent2"]+".", axis=1)

In [ ]:
wordnet_data.head()

,sent1,sent2,label,text
0,abject poverty,the most abject slaves joined in the revolt,1,abject poverty. [SEP] the most abject slaves j...
1,an able seaman,every able-bodied young man served in the army,1,an able seaman. [SEP] every able-bodied young ...
2,able to swim,she was able to program her computer,1,able to swim. [SEP] she was able to program he...
3,she was able to program her computer,we were at last able to buy a car,1,she was able to program her computer. [SEP] we...
4,able to get a grant for the project,we were at last able to buy a car,1,able to get a grant for the project. [SEP] we ...


In [ ]:
wordnet_data.to_csv("./wordnet_train.tsv",sep="\t", index=False)

# **Model**

In [ ]:
from torch.utils.data import TensorDataset, random_split
from transformers import AlbertConfig, AlbertModel, AlbertTokenizer
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AlbertForSequenceClassification, AdamW, AlbertConfig

import numpy as np

from transformers import get_linear_schedule_with_warmup

In [ ]:
all_data = pd.read_csv("./train_data.tsv", sep="\t")
wordnet_data = pd.read_csv("./wordnet_train.tsv", sep="\t")
validation_data = pd.read_csv("./validation_data.tsv", sep="\t")
print(all_data.shape)
all_data = all_data.append(wordnet_data)
all_data.shape

(5428, 8)


(40095, 8)

In [ ]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

In [ ]:
device = "cpu"

if torch.cuda.is_available():
    device = "cuda"

device

'cuda'

In [ ]:
def prepare_input_data(all_data):
    
    tokenized_text = []
    attention_masks = []
    labels = []

    for sentence in all_data["text"].tolist():
        encoded_dict = tokenizer.encode_plus(sentence, add_special_tokens = True, max_length = 64, pad_to_max_length = True,\
                                             return_attention_mask = True, return_tensors = 'pt')

        tokenized_text.append(encoded_dict['input_ids'])    
        attention_masks.append(encoded_dict['attention_mask'])

    tokenized_text = torch.cat(tokenized_text, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(all_data["label"].tolist())
    
    return tokenized_text, attention_masks, labels

train_tokenized_text, train_attention_masks, train_labels = prepare_input_data(all_data)
val_tokenized_text, val_attention_masks, val_labels = prepare_input_data(validation_data)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2217: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:

batch_size = 32
epochs = 10
learning_rate = 0.00002
model_name = "albert-base-v2"

train_dataset = TensorDataset(train_tokenized_text, train_attention_masks, train_labels)
val_dataset = TensorDataset(val_tokenized_text, val_attention_masks, val_labels)

train_dataloader = DataLoader(train_dataset,  sampler = RandomSampler(train_dataset), batch_size = batch_size )
validation_dataloader = DataLoader(val_dataset, sampler = SequentialSampler(val_dataset), batch_size = batch_size )

model = AlbertForSequenceClassification.from_pretrained(model_name, num_labels = 2)
optimizer = optim.Adam(model.parameters(),lr = learning_rate)


Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.bias', 'predictions.bias', 'predictions.dense.bias', 'predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

In [ ]:
## Basic training loop

t0 = time.time()
def get_logit_classification_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

class Train():
    def __init__(self,model, optimiser, train_iterator, validation_iterator,  device="cuda",epochs=5):
        print("program execution start: {}".format(datetime.datetime.now()))
        self.model = model
        self.optimiser = optimiser
        self.best_val_acc = None
        self.epochs = epochs
        self.train_iterator = train_iterator
        self.validation_iterator = validation_iterator
        self.scheduler = get_linear_schedule_with_warmup(self.optimiser, num_warmup_steps = 0, num_training_steps = len(self.train_iterator) * self.epochs)
        # self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimiser, step_size=5, gamma=0.5)
        self.device=device
        self.model.to(self.device)

        print("resource preparation done: {}".format(datetime.datetime.now()))

    def result_checkpoint(self, epoch, train_loss, val_loss, train_acc, val_acc, took):
        if self.best_val_acc is None or val_acc > self.best_val_acc:
          self.best_val_acc = val_acc
          torch.save({
            'accuracy': self.best_val_acc,
            'model_dict': self.model.state_dict(),
          }, '/content/model/weight_params.pt')
        print('| Epoch {:3d} | train loss {:5.2f} | train acc {:5.2f} | val loss {:5.2f} | val acc {:5.2f} | time: {:5.2f}s |'
            .format(epoch, train_loss, train_acc, val_loss, val_acc, took))
  
    def train(self):
        self.model.train(); 
    
        n_total, n_loss, n_acc = 0, 0, 0

        for batch_idx, batch in enumerate(self.train_iterator):
            if(batch_idx%50 == 0):
              print(f"Done for batch {batch_idx} : {time.time()-t0}")
            self.optimiser.zero_grad()
#             torch.cuda.empty_cache()
            
            tokenized_text = batch[0].to(device)
            att_mask = batch[1].to(device)
            labels = batch[2].to(device)

            output = self.model(tokenized_text, attention_mask=att_mask, labels=labels)
          
            logits = output.logits.detach().cpu().numpy()
            label_ids = labels.to('cpu').numpy()

            n_loss += output.loss
            n_acc += get_logit_classification_accuracy(logits, label_ids)
            n_total += len(batch)
            
            output.loss.backward(); 
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimiser.step()
#             self.scheduler.step()

        train_loss = n_loss/n_total
        train_acc = n_acc/len(self.train_iterator)
        return train_loss, train_acc

    def validate(self):
        self.model.eval();
        n_total, n_loss, n_acc = 0, 0, 0
        with torch.no_grad():
            for batch_idx, batch in enumerate(self.validation_iterator):
                tokenized_text = batch[0].to(device)
                att_mask = batch[1].to(device)
                labels = batch[2].to(device)

                output = self.model(tokenized_text, attention_mask=att_mask, labels=labels)
          
                logits = output.logits.detach().cpu().numpy()
                label_ids = labels.to('cpu').numpy()

                n_loss += output.loss
                n_acc += get_logit_classification_accuracy(logits, label_ids)
                n_total += len(batch)
#                 del output
#                 torch.cuda.empty_cache()

        val_loss = n_loss/n_total
        val_acc = n_acc/len(self.validation_iterator)
        return val_loss, val_acc

    def execute(self):
        print(" [*] Training starts!")
        print('-' * 99)
        for epoch in range(1, self.epochs+1):
            start = time.time()

            train_loss, train_acc = self.train()
            val_loss, val_acc = self.validate()
            self.scheduler.step()
            
            took = time.time()-start
            self.result_checkpoint(epoch, train_loss, val_loss, train_acc, val_acc, took)

            print('| Epoch {:3d} | train loss {:6.2f} | train acc {:6.3f} | val loss {:6.3f} | val acc {:6.3f} | time: {:5.2f}s |'.format(
              epoch, train_loss, train_acc, val_loss, round(val_acc,5), took))
        self.finish()

    def finish(self):
        # self.logger.info("[*] Training finished!\n\n")
        print('-' * 99)
        print(" [*] Training finished!")
        print(" [*] Please find the saved model and training log in results_dir")


In [ ]:
t0=time.time()
task = Train(model, optimizer, train_dataloader, validation_dataloader, device, epochs=epochs)
task.execute()

program execution start: 2021-10-25 10:01:12.688163
resource preparation done: 2021-10-25 10:01:12.705639
 [*] Training starts!
---------------------------------------------------------------------------------------------------
Done for batch 0 : 0.02172064781188965
Done for batch 50 : 33.07676100730896
Done for batch 100 : 66.44710779190063
Done for batch 150 : 99.80544853210449
Done for batch 200 : 133.11739587783813
Done for batch 250 : 166.3997824192047
Done for batch 300 : 199.6916823387146
Done for batch 350 : 232.98297667503357
Done for batch 400 : 266.2874939441681
Done for batch 450 : 299.57717394828796
Done for batch 500 : 332.8601531982422
Done for batch 550 : 366.1332778930664
Done for batch 600 : 399.44477915763855
Done for batch 650 : 432.7529876232147
Done for batch 700 : 466.0480296611786
Done for batch 750 : 499.34239315986633
Done for batch 800 : 532.6109800338745
Done for batch 850 : 565.8955442905426
Done for batch 900 : 599.1945734024048
Done for batch 950 : 632.46

In [ ]:
## Zip the final model and all the required files, such as vocabulary
# Replace USERID with your own, such as 2017CSZ8058
!zip -r 2021CSY7548_B_model.zip  /content/model/
## Upload it to Google drive and ensure that the testing notebook uses the correct link

  adding: content/model/ (stored 0%)
  adding: content/model/weight_params.pt (deflated 7%)
